In [24]:
import pandas as pd

In [25]:
patients_df = pd.read_csv("/Users/sri_sai_nikhil_enni/Documents/DMDD/Assignment 3/patients_data.csv")
diagnosis_df = pd.read_csv("/Users/sri_sai_nikhil_enni/Documents/DMDD/Assignment 3/train.csv")
department_df = pd.read_csv("/Users/sri_sai_nikhil_enni/Documents/DMDD/Assignment 3/medicine_prescription_records.csv")
hospitals_df = pd.read_csv("/Users/sri_sai_nikhil_enni/Documents/DMDD/Assignment 3/Hospitals.csv")


### Medicines data

In [26]:
def preprocess_department_data(df):
    
    department_mapper = {"therapeutic radiology" : "radiotherapy",
        "pediatric anesthesiology" : "anesthesia",
        "dentist anesthesiologist": "anesthesia",
        "pulmonary diagnostics" : "chest",
        "pulmonary disease" : "chest",
        'adult reconstructive orthopaedic surgery': 'surgery',
         'facial plastic surgery': 'surgery',
         'female pelvic medicine and reconstructive surgery': 'surgery',
         'foot & ankle surgery': 'surgery',
         'foot surgery': 'surgery',
         'foot and ankle surgery': 'surgery',
         'hand surgery': 'surgery',
         'mohs-micrographic surgery': 'surgery',
         'ophthalmic plastic and reconstructive surgery': 'surgery',
         'ophthalmologic surgery': 'surgery',
         'oral and maxillofacial surgery': 'surgery',
         'orthopaedic surgery of the spine': 'surgery',
         'otolaryngology/facial plastic surgery': 'surgery',
         'pediatric orthopaedic surgery': 'surgery',
         'pediatric surgery': 'surgery',
         'plastic surgery': 'surgery',
         'plastic surgery within the head and neck': 'surgery',
         'plastic surgery within the head & neck': 'surgery',
         'plastic and reconstructive surgery': 'surgery',
         'surgery of the hand': 'surgery',
         'trauma surgery': 'surgery',
         'vascular surgery': 'surgery'}
    
    df = df.rename(columns = {"specialty":"department"})
    df['department'] = df['department'].apply(lambda x : x.lower())
    df['department'] = df['department'].replace(department_mapper)
    
    df = df.groupby('department')['cms_prescription_counts'].apply(lambda x: ','.join(x)).reset_index()

    df['availableMedcines'] = df['cms_prescription_counts'].apply(lambda x : [med[0].lower() + med.title().replace(" ","")[1:] for med in x.split(",")])
    df['availableMedcines'] = df['availableMedcines'].apply(lambda x : [i.strip() for i in x])
    df = df.explode('availableMedcines')
    df.reset_index(drop = True,inplace = True)
    
    return df[["department","availableMedcines"]]

department_df1 = preprocess_department_data(department_df)
# medicines_df1['availableMedcines'] = medicines_df1['availableMedcines'].explode()
department_df1.head()

,department,availableMedcines
0,acute care,spiriva
1,acute care,dvairDiskus
2,acute care,antoprazoleSodium
3,acute care,roairHfa
4,acute care,rednisone


In [27]:
from sklearn.preprocessing import LabelEncoder
le_dep = LabelEncoder()
le_med = LabelEncoder()

medicines_df = pd.DataFrame()

department_df1['department_name'] = department_df1['department']
department_df1['department_id'] = le_dep.fit_transform(department_df1['department'])

medicines_df['medicine_name'] = department_df1['availableMedcines']
department_df1['medicine_id'] = le_med.fit_transform(department_df1['availableMedcines'])
medicines_df['medicine_id'] = department_df1['medicine_id']

department_df1 = department_df1[['department_id','department_name','medicine_id']]

department_df1.head()

,department_id,department_name,medicine_id
0,0,acute care,3100
1,0,acute care,744
2,0,acute care,245
3,0,acute care,2951
4,0,acute care,2840


In [28]:
medicines_df = medicines_df.drop_duplicates().reset_index(drop = True)
medicines_df.head()

,medicine_name,medicine_id
0,spiriva,3100
1,dvairDiskus,744
2,antoprazoleSodium,245
3,roairHfa,2951
4,rednisone,2840


### Hospitals data

In [29]:
hospitals_df.head()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,2017-12-18T00:00:00.000Z,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE


In [30]:
hospitals_df.columns

Index(['X', 'Y', 'OBJECTID', 'ID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
       'ZIP4', 'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY',
       'COUNTYFIPS', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'NAICS_CODE',
       'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE',
       'WEBSITE', 'STATE_ID', 'ALT_NAME', 'ST_FIPS', 'OWNER', 'TTL_STAFF',
       'BEDS', 'TRAUMA', 'HELIPAD'],
      dtype='object')

In [31]:
def preprocess_hospitals_data(df):
    
    for i in ['NAME','ADDRESS','CITY','STATE','ZIP','BEDS']:
        df[i] = df[i].astype(str).apply(lambda x : x.lower())
    
    df = df[['NAME','CITY','ZIP','BEDS','TELEPHONE', 'TYPE', 'STATUS','POPULATION', 'COUNTY','WEBSITE','HELIPAD']]
    df.columns = ['name','city','zipcode','beds','telephone', 'type', 'status','population', 'county','website','helipad']
    df = df.sample(32).reset_index(drop = True).reset_index()
    df = df.rename(columns = {"index" : "hospitalCode"})
    return df

hospitals_df1 = preprocess_hospitals_data(hospitals_df)
hospitals_df1.head()

,hospitalCode,name,city,zipcode,beds,telephone,type,status,population,county,website,helipad
0,0,elkhart general hospital,elkhart,46514,270.0,(574) 294-2621,GENERAL ACUTE CARE,OPEN,270,ELKHART,http://www.egh.org,NOT AVAILABLE
1,1,samaritan lebanon community hospital,lebanon,97355,25.0,NOT AVAILABLE,GENERAL ACUTE CARE,OPEN,25,LINN,www.samhealth.org,Y
2,2,st. james parish hospital,lutcher,70071,20.0,(225) 869-5512,GENERAL ACUTE CARE,OPEN,20,ST. JAMES,http://www.sjph.org,Y
3,3,spectrum health - reed city campus,reed city,49677,25.0,NOT AVAILABLE,CRITICAL ACCESS,OPEN,25,OSCEOLA,http://www.reedcity.spectrum-health.org,NOT AVAILABLE
4,4,mercy medical center,canton,44708,441.0,(330) 489-1000,GENERAL ACUTE CARE,OPEN,441,STARK,http://www.cantonmercy.org,Y


### 3.diagnosis

In [32]:
diagnosis_df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [33]:
le_med = LabelEncoder()

def preprocess_diagnosis_data(df):
    #1.Interval Stay to Integer Stay
    df.loc[df[df['Stay'] == 'More than 100 Days'].index,'Stay'] = "0 - 240"
    df['Stay'] = df['Stay'].apply(lambda x : sum([int(i.strip()) for i in x.split("-")])*0.5)


    
    #3.Renaming columns
    cols = [col.replace("_"," ").title().replace(" ","") for col in df.columns]
    cols = [col[0].lower() + col[1:] for col in cols]
    df.columns = cols
    
    df['department'] = df['department'].apply(lambda x : str(x).lower())
    df['department'] = df['department'].replace({"tb & chest disease" : "chest"})
    df = df.drop(columns = ['hospitalTypeCode','cityCodeHospital','hospitalRegionCode','availableExtraRoomsInHospital','age'])
    return df

diagnosis_df1 = preprocess_diagnosis_data(diagnosis_df)
diagnosis_df1['department_id'] = le_dep.transform(diagnosis_df1['department'])
diagnosis_df1=diagnosis_df1.drop(columns = ['department'])
diagnosis_df1.head()

,caseId,hospitalCode,wardType,wardFacilityCode,bedGrade,patientid,cityCodePatient,typeOfAdmission,severityOfIllness,visitorsWithPatient,admissionDeposit,stay,department_id
0,1,8,R,F,2.0,31397,7.0,Emergency,Extreme,2,4911.0,5.0,221
1,2,2,S,F,2.0,31397,7.0,Trauma,Extreme,2,5954.0,45.5,221
2,3,10,S,E,2.0,31397,7.0,Trauma,Extreme,2,4745.0,35.5,18
3,4,26,R,D,2.0,31397,7.0,Trauma,Extreme,2,7272.0,45.5,221
4,5,26,S,D,2.0,31397,7.0,Trauma,Extreme,2,5558.0,45.5,221


### 1.EDA

In [34]:
# Accuracy: for whatever data described, it needs to be accurate.
#     comments:
        
# Relevancy: the data should meet the requirements for the intended use.
#     comments:
        
# Completeness: the data should not have missing values or miss data records.
#     comments: 
#         Bed Grade                             113
#         City_Code_Patient                    4532
        
#         Above are null value counts. Diagosis - Filling with "null" value 
                
# Consistency:the data should have the data format as expected and can be cross reference-able with the same results.
#     comments:
#         Age and Stay have interval values
#         Diagnosis - Mean of the interval taken


In [35]:
import sqlite3
conn = sqlite3.connect('medis.db')
patients_df.to_sql(name="Patients", con=conn, if_exists='replace', index=False)
diagnosis_df1.to_sql(name="Diagnosis", con=conn, if_exists='replace', index=False)
department_df1.to_sql(name="Department", con=conn, if_exists='replace', index=False)
hospitals_df1.to_sql(name="Hospitals", con=conn, if_exists='replace', index=False)


32